In [2]:
import cvxpy as cp

In [2]:
# Create two scalar optimization variables.
x = cp.Variable()
y = cp.Variable()

In [6]:
# Create two constraints.
constraints = [x + y == 1,
               x - y >= 1]

In [4]:
# Form objective
obj = cp.Minimize((x-y)**2)

In [7]:
# Form and solve problem
prob = cp.Problem(obj, constraints)
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x.value, y.value)

status: optimal
optimal value 1.0
optimal var 1.0 3.50057026691824e-30


In [8]:
# Replace the objective
prob2 = cp.Problem(cp.Maximize(x + y), prob.constraints)
print("optimal value", prob2.solve())


optimal value 0.9999999999999999


In [10]:
# Replace the constraint (x + y == 1)
constraints = [x + y <= 3] + prob2.constraints[1:]
prob3 = cp.Problem(prob2.objective, constraints)
print("optimal value", prob3.solve())

optimal value 2.999999999410368


In [11]:
x = cp.Variable()

# An infeasible problem
prob = cp.Problem(cp.Minimize(x), [x>=1, x<= 0])
prob.solve()
print("status: ", prob.status)
print("optimal value", prob.value)

status:  infeasible
optimal value inf


In [12]:
# An unbounded problem
prob = cp.Problem(cp.Minimize(x))
prob.solve()
print("status: ", prob.status)
print("optimal value", prob.value)

status:  unbounded
optimal value -inf


In [14]:
# A scalar variable
a = cp.Variable()

# Vector variable with shape (5, )
x = cp.Variable(5)

# Column vector variable with shape (5, 1)
x = cp.Variable((5, 1))

# Matrix variable with shape (4, 7)
A = cp.Variable((4, 7))

In [15]:
import numpy as np

# Problem data
m = 10
n = 5
np.random.seed(1)
A = np.random.rand(m,n)
b = np.random.rand(m)

# Construct the problem
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

print("Optimal objective value", prob.solve())
print("Optimal variable value")
print(x.value)

Optimal objective value 0.4669916162642215
Optimal variable value
[2.41537905e-22 9.34711514e-02 3.33654727e-01 4.18131816e-01
 1.46283341e-02]


In [17]:
# Positive scalar parameter
m = cp.Parameter(nonneg = True)

# Column vector parameter with unknown sign (by default)
c = cp.Parameter(5)

# Matrix parameter with negative entries
G = cp.Parameter((4, 7), nonpos = True)

# Assign a constant value to G
G.value = -np.ones((4, 7))

In [18]:
# Create parameter, then assign value
rho = cp.Parameter(nonneg = True)
rho.value = 2

# Initialize parameter with a value
rho = cp.Parameter(nonneg=True, value=2)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = False

# Problem data.
n = 15
m = 10
np.random.seed(1)
A = np.random.randn(n, m)
b = np.random.randn(n)
# gamma must be nonnegative due to DCP rules.
gamma = cp.Parameter(nonneg=True)

# Construct the problem.
x = cp.Variable(m)
error = cp.sum_squares(A @ x - b)
obj = cp.Minimize(error + gamma*cp.norm(x, 1))
prob = cp.Problem(obj)

# Construct a trade-off curve of ||Ax-b||^2 vs. ||x||_1
sq_penalty = []
l1_penalty = []
x_values = []
gamma_vals = np.logspace(-4, 6)
for val in gamma_vals:
    gamma.value = val
    prob.solve()
    # Use expr.value to get the numerical value of
    # an expression in the problem.
    sq_penalty.append(error.value)
    l1_penalty.append(cp.norm(x, 1).value)
    x_values.append(x.value)

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.figure(figsize=(6,10))

# Plot trade-off curve.
plt.subplot(211)
plt.plot(l1_penalty, sq_penalty)
plt.xlabel(r'\|x\|_1', fontsize=16)
plt.ylabel(r'\|Ax-b\|^2', fontsize=16)
plt.title('Trade-Off Curve for LASSO', fontsize=16)

# Plot entries of x vs. gamma.
plt.subplot(212)
for i in range(m):
    plt.plot(gamma_vals, [xi[i] for xi in x_values])
plt.xlabel(r'\gamma', fontsize=16)
plt.ylabel(r'x_{i}', fontsize=16)
plt.xscale('log')
plt.title(r'\text{Entries of x vs. }\gamma', fontsize=16)

plt.tight_layout()
plt.show()


In [ ]:
from multiprocessing import Pool

# Assign a value to gamma and find the optimal x.
def get_x(gamma_value):
    gamma.value = gamma_value
    result = prob.solve()
    return x.value

# Parallel computation (set to 1 process here).
pool = Pool(processes = 1)
x_values = pool.map(get_x, gamma_vals)

In [7]:
import numpy

X = cp.Variable((5, 4))
A = numpy.ones((3, 5))

# Use expr.shape to get the dimensions.
print("dimensions of X:", X.shape)
print("size of X:", X.size)
print("number of dimensions:", X.ndim)
print("dimensions of sum(X):", cp.sum(X).shape)
print("dimensions of A @ X:", (A @ X).shape)

# ValueError raised for invalid dimensions.
try:
    A + X
except ValueError, e:
    print(e)

SyntaxError: multiple exception types must be parenthesized (737465833.py, line 16)

In [9]:
import numpy
x = cp.Variable()
a = cp.Parameter(nonpos=True)
c = numpy.array([1, -1])

print("sign of x:", x.sign)
print("sign of a:", a.sign)
print("sign of square(x):", cp.square(x).sign)
print("sign of c*a:", (c*a).sign)

sign of x: UNKNOWN
sign of a: NONPOSITIVE
sign of square(x): NONNEGATIVE
sign of c*a: UNKNOWN


In [10]:
x = cp.Variable()
a = cp.Parameter(nonneg=True)

print("curvature of x:", x.curvature)
print("curvature of a:", a.curvature)
print("curvature of square(x):", cp.square(x).curvature)
print("curvature of sqrt(x):", cp.sqrt(x).curvature)

curvature of x: AFFINE
curvature of a: CONSTANT
curvature of square(x): CONVEX
curvature of sqrt(x): CONCAVE


In [ ]:
print("sqrt(1 + square(x)) curvature:",
      cp.sqrt(1 + cp.square(x)).curvature)
print("norm(hstack([1, x]), 2) curvature:",
      cp.norm(cp.hstack([1, x]), 2).curvature)